# PyStan workflow

https://mc-stan.org/users/documentation/case-studies/pystan_workflow.html

In [1]:
import pystan
import stan_utility  # for stan diagnostics on model convergence

## linear regression

In [2]:
# stan model
linear_regression_stan_model = """ 
data {
  int<lower=0> N; // sample size
  vector[N] x; // predictor
  vector[N] y; // outcome
}
parameters {
  real alpha;
  real beta;
  real<lower=0> sigma;
}
model {
  y ~ normal(alpha + beta * x, sigma);
  alpha ~ normal(0, 20); // Prior for alpha
  beta ~ normal(1, 10); // Prior for beta
  sigma ~ uniform(0,10); // Prior for sigma
}
"""

# compiling the model
lr_model = pystan.StanModel(model_code=linear_regression_stan_model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_f4ff4b176c62ab5707905172a2eac0db NOW.
/home/wolle/anaconda3/envs/data_science/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /tmp/tmp1k3u9fz1/stanfit4anon_model_f4ff4b176c62ab5707905172a2eac0db_4261036396647070039.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [3]:
# observed data 
lin_dat = {'N': 10,
           'y': [0.1, 1.1, 1.9, 3, 4.2, 5, 6.6, 7, 7.1, 9.9],
           'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}

# fitting the model to the data / calculating the posterior
fit = lr_model.sampling(data=lin_dat, iter=1000, chains=4, seed=42)

In [4]:
print(fit)

Inference for Stan model: anon_model_f4ff4b176c62ab5707905172a2eac0db.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha   0.03    0.02    0.4   -0.8   -0.2   0.04   0.26   0.93    526    1.0
beta    1.01  3.3e-3   0.07   0.86   0.97   1.01   1.06   1.17    532    1.0
sigma   0.59  7.2e-3   0.19   0.35   0.47   0.55   0.68   1.08    665    1.0
lp__    0.51    0.08   1.53  -3.55  -0.16   0.93   1.63   2.24    346   1.01

Samples were drawn using NUTS at Thu Jun 20 19:31:17 2019.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


## Validating the fit

In [6]:
help(stan_utility)

Help on package stan_utility:

NAME
    stan_utility

PACKAGE CONTENTS
    __version__
    file_utils
    save_fit
    stan_generator
    stan_utility
    utils

CLASSES
    builtins.object
        stan_utility.save_fit.StanSavedFit
    
    class StanSavedFit(builtins.object)
     |  StanSavedFit(file_name)
     |  
     |  Methods defined here:
     |  
     |  __init__(self, file_name)
     |      Load a Stan fit from an HDF5 file created by stanfit_to_hdf5.
     |      The parameters are accessed as class attributes.
     |      :param file_name: The file name of the saved stan fit
     |  
     |  display(self)
     |      Display the properties of the stan fit parameters
     |  
     |  ----------------------------------------------------------------------
     |  Data descriptors defined here:
     |  
     |  __dict__
     |      dictionary for instance variables (if defined)
     |  
     |  __weakref__
     |      list of weak references to the object (if defined)

FUNCTIONS

In [7]:
stan_utility.check_all_diagnostics(fit)

n_eff / iter looks reasonable for all parameters
Rhat looks reasonable for all parameters
0.0 of 2000 iterations ended with a divergence (0.0%)
0 of 2000 iterations saturated the maximum tree depth of 10 (0.0%)
E-BFMI indicated no pathological behavior
